<a href="https://colab.research.google.com/github/YapingWu/GoogleColab/blob/main/genpass/evaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

说明：

1.   上传tokenizer和lstm模型



In [8]:
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np
import pandas as pd
import math

# PL模型评估

In [9]:
max_lengths = {
    'myspace': 35,
    'phpbb': 21,
}

In [10]:
def sample(preds, temperature=1.0):
  index = np.random.choice(np.arange(len(preds)), p=preds)
  prop = preds[index]
  return [index, prop]

In [11]:
def get_preds(pl_model, seed_text_list):
  encoded = tokenizer.texts_to_sequences(seed_text_list)  # 对初始文本进行编码
  encoded = pad_sequences(encoded, maxlen=max_length-1, padding='pre')
  preds = pl_model.predict(encoded, batch_size=128)

  rst_list = [sample(pred) for pred in preds]
  result = pd.DataFrame(rst_list, columns=['index', 'prop'])
  result['unit'] = result['index'].apply(lambda x: tokenizer.index_word[x].upper())

  return result

In [12]:
def generating(pl_model, filename, cnt=100):
    # 使用模型生成密码
    print('使用PL模型生成{:,}个wordlist：'.format(cnt))
    units = [''] * cnt
    gen_word_list = []
    while cnt > 0:
        preds = get_preds(pl_model, units)
        concat_unit = np.char.add(units, [' '] * cnt)
        concat_unit = np.char.add(concat_unit, preds['unit'])

        new_units = []
        for unit in concat_unit:
            if '<END>' in unit:
                gen_word_list.append(unit)
            else:
                new_units.append(unit)

        # print('{:,}'.format(len(gen_word_list)))
        units = new_units
        cnt = len(units)

    gen_word_list = np.char.replace(gen_word_list, '<END>', '')  # 删除末尾的'<END>'
    gen_word_list = np.char.strip(gen_word_list)  # 去除前后空格

    filename = '{}{}'.format('./', filename)
    print('生成好的wordlist保存到文件：{}'.format(filename))
    np.savetxt(filename, gen_word_list, fmt='%s', delimiter='\n')

## one-site test

### myspace

In [13]:
data_name = 'myspace'
with open('/content/{}.pkl'.format(data_name), 'rb') as file:
    tokenizer = pickle.load(file)
max_length = max_lengths[data_name]

In [ ]:
# 测试 get_preds
# pl_model = load_model('/content/{}.h5'.format(data_name))
# units = [''] * 100
# preds = get_preds(pl_model, units)
# print(preds.head(5))

In [ ]:
pl_model = load_model('/content/{}.h5'.format(data_name))  # 使用全部数据训练的模型
for alpha in [3, 4, 5, 6, 7, 8, 9]:
    generating(pl_model, '{}{}.txt'.format(data_name, alpha), cnt=int(math.pow(10, alpha)))

使用PL模型生成1,000个wordlist：
生成好的wordlist保存到文件：./myspace3.txt
使用PL模型生成10,000个wordlist：
生成好的wordlist保存到文件：./myspace4.txt
使用PL模型生成100,000个wordlist：
生成好的wordlist保存到文件：./myspace5.txt
使用PL模型生成1,000,000个wordlist：
生成好的wordlist保存到文件：./myspace6.txt
使用PL模型生成10,000,000个wordlist：


In [ ]:
pl_model = load_model('/content/{}_part.h5'.format('data_name'))  # 使用部分数据训练的模型
for alpha in [3, 4, 5, 6, 7, 8, 9]:
    generating(pl_model, '{}_{}_part.txt'.format(data_name, alpha), cnt=int(math.pow(10, alpha)))

### phpbb

In [ ]:
data_name = 'myspace'
with open('/content/{}.pkl'.format(data_name), 'rb') as file:
    tokenizer = pickle.load(file)
max_length = max_lengths[data_name]

In [ ]:
pl_model = load_model('/content/{}.h5'.format(data_name))  # 使用全部数据训练的模型
for alpha in [3, 4, 5, 6, 7, 8, 9]:
    generating(pl_model, '{}{}.txt'.format(data_name, alpha), cnt=int(math.pow(10, alpha)))

In [ ]:
pl_model = load_model('/content/{}_part.h5'.format('data_name'))  # 使用部分数据训练的模型
for alpha in [3, 4, 5, 6, 7, 8, 9]:
    generating(pl_model, '{}_{}_part.txt'.format(data_name, alpha), cnt=int(math.pow(10, alpha)))

## cross-site test

# GenPass模型评估

# 其他命令

## 解压文件

In [7]:
!unzip '/content/lstm_1.zip'
!unzip '/content/tokenizer.zip'

Archive:  /content/lstm_1.zip
  inflating: phpbb.h5                
  inflating: phpbb_part.h5           
  inflating: myspace.h5              
  inflating: myspace_part.h5         
Archive:  /content/tokenizer.zip
  inflating: phpbb.pkl               
  inflating: rockyou.pkl             
  inflating: myspace.pkl             
